# Building flat unstructured geometries

In [ ]:
import pycpt
import lavavu

import meshio
import stripy
import numpy as np
import pygmsh as pg

from scipy.interpolate import RectBivariateSpline

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

## Using pyGmsh library

Using `pyGmsh` library, which is a Python interface to `Gmsh`: a powerful mesh generation tool:

- https://github.com/nschloe/pygmsh

The resolution of the mesh is defined based on the characteristic length parameter `lcar`.

There are several different ways of defining the mesh, here we use an approach similar to what we've done previously with `stripy` and use the border coordinates to create our delaunay triangulation.

### Bounding box

We first initialise a pyGmsh instance and define the characteristc length `lcar`:

In [ ]:
minX, maxX = 0.0, 100000.0
minY, maxY = 0.0, 100000.0

lcar = 200
geom1 = pg.built_in.Geometry()

We then define the polygon based on the box extent:

In [ ]:
poly = geom1.add_polygon([
    [ minX,  minY, 0.0],
    [ maxX,  minY, 0.0],
    [ maxX,  maxY, 0.0],
    [ minX,  maxY, 0.0]
    ],
    lcar
    )

We then generate the mesh...

In [ ]:
points, cells, _, _, _ = pg.generate_mesh(geom1)

In [ ]:
print("Unstructured Grid Geometry Characteristics:\n")

print("Number of points         nbpt: {}".format(points.shape[0]))
print("Number of faces       nbcells: {}".format(cells['triangle'].shape[0]))

# Random elevation

To create the elevation grids not much needs to be done. Basically we will create an array XYZ in numpy that we will write to the disk.

In [ ]:
randomZ = np.random.rand(points.shape[0])

We use `meshio` to create a VTK file that will then be passed to the landscape evolution model.

In [ ]:
# Creation of the X,Y,Z coordinates of the unstructured grid
verts = np.insert(points[:,:2], 2, values=randomZ, axis=1)

In [ ]:
mesh = meshio.Mesh(verts, cells, {'Z':randomZ})
meshio.write("randomflat.vtk", mesh)